# Setup

This is following [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html) for pandas 1.4.3

In [1]:
import numpy as np
import pandas as pd

# Object creation

In [2]:
# Series - pass a list to pd.Series
s1 = pd.Series([1, 2, 3, np.nan, 5])

In [3]:
s1

0    1.0
1    2.0
2    3.0
3    NaN
4    5.0
dtype: float64

In [4]:
# create an index to use in the dataframe
# default frequency is D (day), so creates a DatetimeIndex of 10 days
dates = pd.date_range('20220619', periods=10)
dates

DatetimeIndex(['2022-06-19', '2022-06-20', '2022-06-21', '2022-06-22',
               '2022-06-23', '2022-06-24', '2022-06-25', '2022-06-26',
               '2022-06-27', '2022-06-28'],
              dtype='datetime64[ns]', freq='D')

just in case you are wondering, here's [all the "offset aliases"](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases) for Datetime

In [5]:
# Dataframe - pass a Numpy array
# create a 10 row, 4 col random number array, index by dates, give some column names
df = pd.DataFrame(np.random.randn(10, 4), index=dates, columns=list('ABCD'))
df

,A,B,C,D
2022-06-19,-0.971039,-1.302320,-1.163174,-1.641720
2022-06-20,-1.018621,0.719277,-0.352777,-0.025486
2022-06-21,0.023506,0.267982,0.354560,-1.246931
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176
2022-06-23,-0.300175,-1.309658,0.473513,-1.316474
2022-06-24,2.108041,0.070278,-0.961669,-1.211711
2022-06-25,-0.785940,-1.622560,-1.594312,0.221272
2022-06-26,-0.523616,-0.117206,0.189113,-1.471392
2022-06-27,0.163592,1.227369,-0.486297,-1.283956
2022-06-28,-1.222419,-0.619103,-1.656902,-0.494676


In [6]:
# Create a dataframe by passing a dictionary of objects
# where each object can be converted into a series-like structure
df2 = pd.DataFrame(
    {
        "A": 1.0,
        "B": pd.Timestamp("20220619"),
        "C": pd.Series(1, index=list(range(6)), dtype="float32"),
        "D": np.array([3]*6, dtype="int32"),
        "E": pd.Categorical(["test", "train", "test", "train", "test", "train"]),
        "F": list("foofoo"),
        "G": "foo"
    }
)

df2

,A,B,C,D,E,F,G
0,1.0,2022-06-19,1.0,3,test,f,foo
1,1.0,2022-06-19,1.0,3,train,o,foo
2,1.0,2022-06-19,1.0,3,test,o,foo
3,1.0,2022-06-19,1.0,3,train,f,foo
4,1.0,2022-06-19,1.0,3,test,o,foo
5,1.0,2022-06-19,1.0,3,train,o,foo


In [7]:
# the datatype of each of the columns would be different
df2.dtypes

A           float64
B    datetime64[ns]
C           float32
D             int32
E          category
F            object
G            object
dtype: object

# Viewing Data

In [8]:
# top of the dataframe
df.head()

,A,B,C,D
2022-06-19,-0.971039,-1.302320,-1.163174,-1.641720
2022-06-20,-1.018621,0.719277,-0.352777,-0.025486
2022-06-21,0.023506,0.267982,0.354560,-1.246931
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176
2022-06-23,-0.300175,-1.309658,0.473513,-1.316474


In [9]:
# bottom 3 records of the dataframe
df.tail(3)

,A,B,C,D
2022-06-26,-0.523616,-0.117206,0.189113,-1.471392
2022-06-27,0.163592,1.227369,-0.486297,-1.283956
2022-06-28,-1.222419,-0.619103,-1.656902,-0.494676


In [10]:
# index of the df
df.index

DatetimeIndex(['2022-06-19', '2022-06-20', '2022-06-21', '2022-06-22',
               '2022-06-23', '2022-06-24', '2022-06-25', '2022-06-26',
               '2022-06-27', '2022-06-28'],
              dtype='datetime64[ns]', freq='D')

In [11]:
# columns of the df
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [12]:
# .to_numpy gives a NumPy representation of the dataframe
# this is expensive if all columns are of different data type
df.to_numpy()

array([[-0.97103935, -1.30231963, -1.16317381, -1.6417197 ],
       [-1.01862136,  0.71927743, -0.35277708, -0.02548558],
       [ 0.0235062 ,  0.26798224,  0.35455963, -1.246931  ],
       [-0.67136993, -0.80209922, -0.85097525,  0.28317588],
       [-0.30017546, -1.30965814,  0.4735131 , -1.31647351],
       [ 2.10804124,  0.07027844, -0.96166943, -1.21171109],
       [-0.78593981, -1.62256038, -1.59431181,  0.22127197],
       [-0.52361568, -0.11720581,  0.1891131 , -1.47139158],
       [ 0.16359157,  1.22736907, -0.48629658, -1.2839563 ],
       [-1.22241893, -0.61910264, -1.65690179, -0.49467554]])

In [13]:
df2.to_numpy()

array([[1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'test', 'f',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'train', 'o',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'test', 'o',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'train', 'f',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'test', 'o',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'train', 'o',
        'foo']], dtype=object)

In [14]:
# quick summary stats
df.describe()

,A,B,C,D
count,10.000000,10.000000,10.000000,10.000000
mean,-0.319804,-0.348804,-0.604892,-0.818790
std,0.963555,0.940936,0.772929,0.740518
min,-1.222419,-1.622560,-1.656902,-1.641720
25%,-0.924764,-1.177265,-1.112798,-1.308344
50%,-0.597493,-0.368154,-0.668636,-1.229321
75%,-0.057414,0.218556,0.053641,-0.142783
max,2.108041,1.227369,0.473513,0.283176


1. **count** = Count number of non-NA/null observations
1. **max** = Maximum of the values in the object
1. **min** = Minimum of the values in the object
1. **mean** = Mean of the values
1. **std** = Standard deviation of the observations
1. **25%** = Default lower percentile
1. **50%** = 50 percentile - same as the median 
1. **75%** = Default upper percentile

In [15]:
# change the percentiles
df.describe(percentiles=[.1, .5, .9])

,A,B,C,D
count,10.000000,10.000000,10.000000,10.000000
mean,-0.319804,-0.348804,-0.604892,-0.818790
std,0.963555,0.940936,0.772929,0.740518
min,-1.222419,-1.622560,-1.656902,-1.641720
10%,-1.039001,-1.340948,-1.600571,-1.488424
50%,-0.597493,-0.368154,-0.668636,-1.229321
90%,0.358037,0.770087,0.366455,0.227462
max,2.108041,1.227369,0.473513,0.283176


In [16]:
# E, F, G in d2 are not numeric, so do not come up in describe()
df2.describe()

,A,C,D
count,6.0,6.0,6.0
mean,1.0,1.0,3.0
std,0.0,0.0,0.0
min,1.0,1.0,3.0
25%,1.0,1.0,3.0
50%,1.0,1.0,3.0
75%,1.0,1.0,3.0
max,1.0,1.0,3.0


In [17]:
# transpose the data
# turn rows to columns and vice versa
df.T

,2022-06-19,2022-06-20,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28
A,-0.971039,-1.018621,0.023506,-0.671370,-0.300175,2.108041,-0.785940,-0.523616,0.163592,-1.222419
B,-1.302320,0.719277,0.267982,-0.802099,-1.309658,0.070278,-1.622560,-0.117206,1.227369,-0.619103
C,-1.163174,-0.352777,0.354560,-0.850975,0.473513,-0.961669,-1.594312,0.189113,-0.486297,-1.656902
D,-1.641720,-0.025486,-1.246931,0.283176,-1.316474,-1.211711,0.221272,-1.471392,-1.283956,-0.494676


### [Axis in pandas](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_axis.html?highlight=set_ax#pandas.DataFrame.set_axis)

```axis{0 or ‘index’, 1 or ‘columns’}```

Always remember that when you specify:
* ```axis=0``` or ```axis='index'``` you indicate that the operation should be along the _index_ (aka across the rows). More often than not indexes will go from top to bottom (vertically), but sometimes they may not (for e.g. if you use columns as index or when you have hierarchical or multi-index data).
* ```axis=1``` or ```axis='columns'``` indicates the operation is along the columns

When in doubt use the explicit version ```'index'``` or ```'columns'``` instead of ```0``` or ```1```

In [18]:
# sort along the axis - 1 = horizontal
df.sort_index(axis=1, ascending=False)

,D,C,B,A
2022-06-19,-1.641720,-1.163174,-1.302320,-0.971039
2022-06-20,-0.025486,-0.352777,0.719277,-1.018621
2022-06-21,-1.246931,0.354560,0.267982,0.023506
2022-06-22,0.283176,-0.850975,-0.802099,-0.671370
2022-06-23,-1.316474,0.473513,-1.309658,-0.300175
2022-06-24,-1.211711,-0.961669,0.070278,2.108041
2022-06-25,0.221272,-1.594312,-1.622560,-0.785940
2022-06-26,-1.471392,0.189113,-0.117206,-0.523616
2022-06-27,-1.283956,-0.486297,1.227369,0.163592
2022-06-28,-0.494676,-1.656902,-0.619103,-1.222419


In [19]:
# sort along the axis - 0 = vertical
df.sort_index(axis=0, ascending=False)

,A,B,C,D
2022-06-28,-1.222419,-0.619103,-1.656902,-0.494676
2022-06-27,0.163592,1.227369,-0.486297,-1.283956
2022-06-26,-0.523616,-0.117206,0.189113,-1.471392
2022-06-25,-0.785940,-1.622560,-1.594312,0.221272
2022-06-24,2.108041,0.070278,-0.961669,-1.211711
2022-06-23,-0.300175,-1.309658,0.473513,-1.316474
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176
2022-06-21,0.023506,0.267982,0.354560,-1.246931
2022-06-20,-1.018621,0.719277,-0.352777,-0.025486
2022-06-19,-0.971039,-1.302320,-1.163174,-1.641720


In [20]:
# sort ascending by values in a column
df.sort_values(by="A")

,A,B,C,D
2022-06-28,-1.222419,-0.619103,-1.656902,-0.494676
2022-06-20,-1.018621,0.719277,-0.352777,-0.025486
2022-06-19,-0.971039,-1.302320,-1.163174,-1.641720
2022-06-25,-0.785940,-1.622560,-1.594312,0.221272
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176
2022-06-26,-0.523616,-0.117206,0.189113,-1.471392
2022-06-23,-0.300175,-1.309658,0.473513,-1.316474
2022-06-21,0.023506,0.267982,0.354560,-1.246931
2022-06-27,0.163592,1.227369,-0.486297,-1.283956
2022-06-24,2.108041,0.070278,-0.961669,-1.211711


In [21]:
# sort by non-numerical values
df2.sort_values(by="F", ascending=False)

,A,B,C,D,E,F,G
1,1.0,2022-06-19,1.0,3,train,o,foo
2,1.0,2022-06-19,1.0,3,test,o,foo
4,1.0,2022-06-19,1.0,3,test,o,foo
5,1.0,2022-06-19,1.0,3,train,o,foo
0,1.0,2022-06-19,1.0,3,test,f,foo
3,1.0,2022-06-19,1.0,3,train,f,foo


In [22]:
# sort by two or more columns
df2.sort_values(by=["F", "E"])

,A,B,C,D,E,F,G
0,1.0,2022-06-19,1.0,3,test,f,foo
3,1.0,2022-06-19,1.0,3,train,f,foo
2,1.0,2022-06-19,1.0,3,test,o,foo
4,1.0,2022-06-19,1.0,3,test,o,foo
1,1.0,2022-06-19,1.0,3,train,o,foo
5,1.0,2022-06-19,1.0,3,train,o,foo


# Selection

For production prefer the following instead of other data access methods (typical python methods like ["col"] or [a:b] slices etc.):

```.at, .iat, .loc and .iloc.```

## Getting

In [23]:
# selecting a single column returns a Series object
df["A"]

2022-06-19   -0.971039
2022-06-20   -1.018621
2022-06-21    0.023506
2022-06-22   -0.671370
2022-06-23   -0.300175
2022-06-24    2.108041
2022-06-25   -0.785940
2022-06-26   -0.523616
2022-06-27    0.163592
2022-06-28   -1.222419
Freq: D, Name: A, dtype: float64

In [24]:
# selecting a slice
df[1:5]

,A,B,C,D
2022-06-20,-1.018621,0.719277,-0.352777,-0.025486
2022-06-21,0.023506,0.267982,0.354560,-1.246931
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176
2022-06-23,-0.300175,-1.309658,0.473513,-1.316474


## Selection by label

In [25]:
# selecting based on a label
df.loc[dates[0]]

A   -0.971039
B   -1.302320
C   -1.163174
D   -1.641720
Name: 2022-06-19 00:00:00, dtype: float64

In [26]:
# select on a multi-axis by lable
# I honestly do not know what the comment above means...help!
df.loc[:, ["A", "B"]]

,A,B
2022-06-19,-0.971039,-1.302320
2022-06-20,-1.018621,0.719277
2022-06-21,0.023506,0.267982
2022-06-22,-0.671370,-0.802099
2022-06-23,-0.300175,-1.309658
2022-06-24,2.108041,0.070278
2022-06-25,-0.785940,-1.622560
2022-06-26,-0.523616,-0.117206
2022-06-27,0.163592,1.227369
2022-06-28,-1.222419,-0.619103


In [27]:
# specific index value results in reduction of dimensions
res = df.loc["2022-06-20"]
print(res)
print("res.shape = ", res.shape, " vs. df.shape = ", df.shape)

A   -1.018621
B    0.719277
C   -0.352777
D   -0.025486
Name: 2022-06-20 00:00:00, dtype: float64
res.shape =  (4,)  vs. df.shape =  (10, 4)


In [28]:
# get to a specific scalar:
#
# method one
df.loc[dates[0], "A"]

-0.9710393464426902

In [29]:
#
# method two (slightly faster than method one)
df.at[dates[0], "A"]

-0.9710393464426902

## Selection by position

In [30]:
df.iloc[2]

A    0.023506
B    0.267982
C    0.354560
D   -1.246931
Name: 2022-06-21 00:00:00, dtype: float64

In [31]:
# slices - similar to NumPy / Python - [row:slice, col:slice]
df.iloc[1:5, 0:2]

,A,B
2022-06-20,-1.018621,0.719277
2022-06-21,0.023506,0.267982
2022-06-22,-0.671370,-0.802099
2022-06-23,-0.300175,-1.309658


In [32]:
# by list of locations - similar to NumPy / Python - [[list of rows], [list of cols]]
df.iloc[[0, 1, 2, 6], [0, 2]]

,A,C
2022-06-19,-0.971039,-1.163174
2022-06-20,-1.018621,-0.352777
2022-06-21,0.023506,0.354560
2022-06-25,-0.785940,-1.594312


In [33]:
# by list of locations - similar to NumPy / Python - [[list of rows], [list of cols]]
# change the order of columns, repeact a column
df.iloc[[0, 1, 2, 6], [2, 1, 0, 2]]

,C,B,A,C
2022-06-19,-1.163174,-1.302320,-0.971039,-1.163174
2022-06-20,-0.352777,0.719277,-1.018621,-0.352777
2022-06-21,0.354560,0.267982,0.023506,0.354560
2022-06-25,-1.594312,-1.622560,-0.785940,-1.594312


In [34]:
# slice rows explicitly, keep all columns
df.iloc[[1, 2], :]

,A,B,C,D
2022-06-20,-1.018621,0.719277,-0.352777,-0.025486
2022-06-21,0.023506,0.267982,0.354560,-1.246931


In [35]:
# slice columns, keep all rows
df.iloc[:, [2, 3]]

,C,D
2022-06-19,-1.163174,-1.641720
2022-06-20,-0.352777,-0.025486
2022-06-21,0.354560,-1.246931
2022-06-22,-0.850975,0.283176
2022-06-23,0.473513,-1.316474
2022-06-24,-0.961669,-1.211711
2022-06-25,-1.594312,0.221272
2022-06-26,0.189113,-1.471392
2022-06-27,-0.486297,-1.283956
2022-06-28,-1.656902,-0.494676


In [36]:
# everything, because you can
df.iloc[:, :]

,A,B,C,D
2022-06-19,-0.971039,-1.302320,-1.163174,-1.641720
2022-06-20,-1.018621,0.719277,-0.352777,-0.025486
2022-06-21,0.023506,0.267982,0.354560,-1.246931
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176
2022-06-23,-0.300175,-1.309658,0.473513,-1.316474
2022-06-24,2.108041,0.070278,-0.961669,-1.211711
2022-06-25,-0.785940,-1.622560,-1.594312,0.221272
2022-06-26,-0.523616,-0.117206,0.189113,-1.471392
2022-06-27,0.163592,1.227369,-0.486297,-1.283956
2022-06-28,-1.222419,-0.619103,-1.656902,-0.494676


In [37]:
# get to a scalar (2 methods, just like before)
#
# method one: use iloc
df.iloc[1, 2]

-0.3527770812342945

In [38]:
#
# method two: use iat
df.iat[1, 2]

-0.3527770812342945

## Boolean Indexing

In [39]:
# use a value found in a single col to get data
df[df["A"] > 0]

,A,B,C,D
2022-06-21,0.023506,0.267982,0.354560,-1.246931
2022-06-24,2.108041,0.070278,-0.961669,-1.211711
2022-06-27,0.163592,1.227369,-0.486297,-1.283956


In [40]:
# boolean across the entire DF - vals that don't match go NaN
df[df > 0]

,A,B,C,D
2022-06-19,NaN,NaN,NaN,NaN
2022-06-20,NaN,0.719277,NaN,NaN
2022-06-21,0.023506,0.267982,0.354560,NaN
2022-06-22,NaN,NaN,NaN,0.283176
2022-06-23,NaN,NaN,0.473513,NaN
2022-06-24,2.108041,0.070278,NaN,NaN
2022-06-25,NaN,NaN,NaN,0.221272
2022-06-26,NaN,NaN,0.189113,NaN
2022-06-27,0.163592,1.227369,NaN,NaN
2022-06-28,NaN,NaN,NaN,NaN


In [41]:
# add another column
df11 = df.copy()
df11["E"] = ["one", "two", "three", "four",
             "two", "five", "one", "two", "three", "four"]
df11

,A,B,C,D,E
2022-06-19,-0.971039,-1.302320,-1.163174,-1.641720,one
2022-06-20,-1.018621,0.719277,-0.352777,-0.025486,two
2022-06-21,0.023506,0.267982,0.354560,-1.246931,three
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176,four
2022-06-23,-0.300175,-1.309658,0.473513,-1.316474,two
2022-06-24,2.108041,0.070278,-0.961669,-1.211711,five
2022-06-25,-0.785940,-1.622560,-1.594312,0.221272,one
2022-06-26,-0.523616,-0.117206,0.189113,-1.471392,two
2022-06-27,0.163592,1.227369,-0.486297,-1.283956,three
2022-06-28,-1.222419,-0.619103,-1.656902,-0.494676,four


In [42]:
# the isin() query - basically the in clause
df11[df11["E"].isin(["two", "five"])]

,A,B,C,D,E
2022-06-20,-1.018621,0.719277,-0.352777,-0.025486,two
2022-06-23,-0.300175,-1.309658,0.473513,-1.316474,two
2022-06-24,2.108041,0.070278,-0.961669,-1.211711,five
2022-06-26,-0.523616,-0.117206,0.189113,-1.471392,two


## Setting values

In [43]:
# matching indexes auto-aligns values
s1 = pd.Series(range(11, 21), index=pd.date_range('20220619', periods=10))
s1

2022-06-19    11
2022-06-20    12
2022-06-21    13
2022-06-22    14
2022-06-23    15
2022-06-24    16
2022-06-25    17
2022-06-26    18
2022-06-27    19
2022-06-28    20
Freq: D, dtype: int64

In [44]:
df["F"] = s1
df

,A,B,C,D,F
2022-06-19,-0.971039,-1.302320,-1.163174,-1.641720,11
2022-06-20,-1.018621,0.719277,-0.352777,-0.025486,12
2022-06-21,0.023506,0.267982,0.354560,-1.246931,13
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176,14
2022-06-23,-0.300175,-1.309658,0.473513,-1.316474,15
2022-06-24,2.108041,0.070278,-0.961669,-1.211711,16
2022-06-25,-0.785940,-1.622560,-1.594312,0.221272,17
2022-06-26,-0.523616,-0.117206,0.189113,-1.471392,18
2022-06-27,0.163592,1.227369,-0.486297,-1.283956,19
2022-06-28,-1.222419,-0.619103,-1.656902,-0.494676,20


In [45]:
# setting values by label and position
# first let's make a quick copy
df12 = df.copy()
df12

,A,B,C,D,F
2022-06-19,-0.971039,-1.302320,-1.163174,-1.641720,11
2022-06-20,-1.018621,0.719277,-0.352777,-0.025486,12
2022-06-21,0.023506,0.267982,0.354560,-1.246931,13
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176,14
2022-06-23,-0.300175,-1.309658,0.473513,-1.316474,15
2022-06-24,2.108041,0.070278,-0.961669,-1.211711,16
2022-06-25,-0.785940,-1.622560,-1.594312,0.221272,17
2022-06-26,-0.523616,-0.117206,0.189113,-1.471392,18
2022-06-27,0.163592,1.227369,-0.486297,-1.283956,19
2022-06-28,-1.222419,-0.619103,-1.656902,-0.494676,20


In [46]:
# set by label
df12.at[dates[0], "A"] = 0
# set by position
df12.iat[0, 1] = 0
df12

,A,B,C,D,F
2022-06-19,0.000000,0.000000,-1.163174,-1.641720,11
2022-06-20,-1.018621,0.719277,-0.352777,-0.025486,12
2022-06-21,0.023506,0.267982,0.354560,-1.246931,13
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176,14
2022-06-23,-0.300175,-1.309658,0.473513,-1.316474,15
2022-06-24,2.108041,0.070278,-0.961669,-1.211711,16
2022-06-25,-0.785940,-1.622560,-1.594312,0.221272,17
2022-06-26,-0.523616,-0.117206,0.189113,-1.471392,18
2022-06-27,0.163592,1.227369,-0.486297,-1.283956,19
2022-06-28,-1.222419,-0.619103,-1.656902,-0.494676,20


In [47]:
# kinda bigger replacement
df12.loc[:, "D"] = np.array([5]*len(df))
df12

,A,B,C,D,F
2022-06-19,0.000000,0.000000,-1.163174,5,11
2022-06-20,-1.018621,0.719277,-0.352777,5,12
2022-06-21,0.023506,0.267982,0.354560,5,13
2022-06-22,-0.671370,-0.802099,-0.850975,5,14
2022-06-23,-0.300175,-1.309658,0.473513,5,15
2022-06-24,2.108041,0.070278,-0.961669,5,16
2022-06-25,-0.785940,-1.622560,-1.594312,5,17
2022-06-26,-0.523616,-0.117206,0.189113,5,18
2022-06-27,0.163592,1.227369,-0.486297,5,19
2022-06-28,-1.222419,-0.619103,-1.656902,5,20


In [48]:
# setting values using a boolean selection (aka where clause)
df12[df12 > 0] = -df12
df12

,A,B,C,D,F
2022-06-19,0.000000,0.000000,-1.163174,-5,-11
2022-06-20,-1.018621,-0.719277,-0.352777,-5,-12
2022-06-21,-0.023506,-0.267982,-0.354560,-5,-13
2022-06-22,-0.671370,-0.802099,-0.850975,-5,-14
2022-06-23,-0.300175,-1.309658,-0.473513,-5,-15
2022-06-24,-2.108041,-0.070278,-0.961669,-5,-16
2022-06-25,-0.785940,-1.622560,-1.594312,-5,-17
2022-06-26,-0.523616,-0.117206,-0.189113,-5,-18
2022-06-27,-0.163592,-1.227369,-0.486297,-5,-19
2022-06-28,-1.222419,-0.619103,-1.656902,-5,-20


# Missing Data

### _reindex_
change/add/delete index on a specified axis, returns a new dataframe

In [49]:
df13 = df.reindex(index=dates[0:4], columns=list(df.columns)+["G"])
df13

,A,B,C,D,F,G
2022-06-19,-0.971039,-1.302320,-1.163174,-1.641720,11,NaN
2022-06-20,-1.018621,0.719277,-0.352777,-0.025486,12,NaN
2022-06-21,0.023506,0.267982,0.354560,-1.246931,13,NaN
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176,14,NaN


### handling missing data
1. _```np.nan```_
1. _```pandas.isna()```_
1. ```df.dropna()```
1. ```df.fillna()```

In [50]:
# missing data in pandas is np.nan
df13.iat[1, 0] = np.nan
df13.iloc[1:, 5] = np.random.randint(1)
df13

,A,B,C,D,F,G
2022-06-19,-0.971039,-1.302320,-1.163174,-1.641720,11,NaN
2022-06-20,NaN,0.719277,-0.352777,-0.025486,12,0.0
2022-06-21,0.023506,0.267982,0.354560,-1.246931,13,0.0
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176,14,0.0


In [51]:
# get a boolean mask where values are NaN
df131 = pd.isna(df13)
df131

,A,B,C,D,F,G
2022-06-19,False,False,False,False,False,True
2022-06-20,True,False,False,False,False,False
2022-06-21,False,False,False,False,False,False
2022-06-22,False,False,False,False,False,False


In [52]:
# or just
pd.isna(df13)

,A,B,C,D,F,G
2022-06-19,False,False,False,False,False,True
2022-06-20,True,False,False,False,False,False
2022-06-21,False,False,False,False,False,False
2022-06-22,False,False,False,False,False,False


In [53]:
# the original is still there
df13

,A,B,C,D,F,G
2022-06-19,-0.971039,-1.302320,-1.163174,-1.641720,11,NaN
2022-06-20,NaN,0.719277,-0.352777,-0.025486,12,0.0
2022-06-21,0.023506,0.267982,0.354560,-1.246931,13,0.0
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176,14,0.0


In [54]:
# we are going to drop / replace values now, let's make a couple of copies of the dataframe
df132 = df13.copy()
df132

,A,B,C,D,F,G
2022-06-19,-0.971039,-1.302320,-1.163174,-1.641720,11,NaN
2022-06-20,NaN,0.719277,-0.352777,-0.025486,12,0.0
2022-06-21,0.023506,0.267982,0.354560,-1.246931,13,0.0
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176,14,0.0


In [55]:
# drop rows/columns that have missing data
# by default it returns a new dataframe, you may want to specify inplace=True for modifying current dataframe:
df_no_na = df132.dropna(how="any")
# how=‘any’ : If any NA values are present, drop that row or column.
# how=‘all’ : If all values are NA, drop that row or column.

In [56]:
# all rows/cols with missing data stripped
df_no_na

,A,B,C,D,F,G
2022-06-21,0.023506,0.267982,0.354560,-1.246931,13,0.0
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176,14,0.0


In [57]:
# original still intact
df132

,A,B,C,D,F,G
2022-06-19,-0.971039,-1.302320,-1.163174,-1.641720,11,NaN
2022-06-20,NaN,0.719277,-0.352777,-0.025486,12,0.0
2022-06-21,0.023506,0.267982,0.354560,-1.246931,13,0.0
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176,14,0.0


In [58]:
# drop missing data from original
df132.dropna(how="any", inplace=True)
df132

,A,B,C,D,F,G
2022-06-21,0.023506,0.267982,0.354560,-1.246931,13,0.0
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176,14,0.0


In [59]:
# fill missing data
df133 = df13.copy()
df133.fillna(np.pi*1000)

,A,B,C,D,F,G
2022-06-19,-0.971039,-1.302320,-1.163174,-1.641720,11,3141.592654
2022-06-20,3141.592654,0.719277,-0.352777,-0.025486,12,0.000000
2022-06-21,0.023506,0.267982,0.354560,-1.246931,13,0.000000
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176,14,0.000000


# Align and Join

There needs to be a bigger notebook for this topic.
You need to know there's "joins" in Pandas just like in the SQL world, 
like join and left join and right join and inner and outer and all that...
Here we'll explore these in the context of the align function - 'aligning' the indexes of two dataframes.
Later we'll see these again in the context of _merging_ two data frames.

## Create two datasets with _mismatched_ indexes

In [60]:
# date range indexes
idx1 = pd.date_range('2022-01-01',periods = 10)
# 2022-01-01', '2022-01-02' don't exist in idx2
# '2022-01-11', '2022-01-12' don't exist in idx1
idx2 = pd.date_range('2022-01-03',periods = 10)

In [61]:
# dataframes from indexes
d1 = pd.DataFrame(index = idx1, data = 
    {
        'A': np.random.rand(10),
        'B': np.random.randint(1, high=25, size = 10)
    }
)

d2 = pd.DataFrame(index = idx2, data = 
    {
        'A': np.random.rand(10),
        'B': np.random.randint(1, high=25, size = 10)
    }
)

## Quick Aside: Rendering two dataframes side-by-side

The trick of rendering 2 dataframes side-by-side was from [this](https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side) stackoverflow question.

In [62]:
# we gon need to display both data frames side by side, so...
from IPython.display import display_html 
def render_df_side_by_side(a, b, a_title = "", b_title = ""):
    a_styler = a.style.set_table_attributes("style='display:inline'").set_caption(a_title)
    b_styler = b.style.set_table_attributes("style='display:inline'").set_caption(b_title)
    display_html(a_styler._repr_html_()+b_styler._repr_html_(), raw=True)

In [63]:
render_df_side_by_side(d1,d2)

,A,B
2022-01-01 00:00:00,0.915867,7
2022-01-02 00:00:00,0.204894,9
2022-01-03 00:00:00,0.102420,6
2022-01-04 00:00:00,0.109014,2
2022-01-05 00:00:00,0.205643,18
2022-01-06 00:00:00,0.299396,21
2022-01-07 00:00:00,0.690629,23
2022-01-08 00:00:00,0.432113,7
2022-01-09 00:00:00,0.868625,15
2022-01-10 00:00:00,0.746700,5


Table rendering/styling options is [a bigger discussion](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#) to be tackled later.

## .add, .sub, .mul, .div etc.

flexible wrappers (```add, sub, mul, div, mod, pow```) to arithmetic operators: ```+, -, *, /, //, %, **```

In [64]:
# the +, -, *, /, //, %, ** operations align indexes to create a UNION of indexes
d3 = d1+d2
d4 = d1.sub(d2) #equivalent to d1-d2
d5 = d1*d2
d6 = d1.div(d2)

In [65]:
# indexs not present in the other dataframe get a NaN
d3.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,0.483887,17.000000
2022-01-04 00:00:00,0.433941,5.000000
2022-01-05 00:00:00,0.494248,37.000000
2022-01-06 00:00:00,0.723410,29.000000
2022-01-07 00:00:00,1.500265,47.000000
2022-01-08 00:00:00,1.333319,9.000000
2022-01-09 00:00:00,1.682297,27.000000
2022-01-10 00:00:00,1.187141,7.000000


In [66]:
# sub
d4.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,-0.279047,-5.000000
2022-01-04 00:00:00,-0.215913,-1.000000
2022-01-05 00:00:00,-0.082962,-1.000000
2022-01-06 00:00:00,-0.124617,13.000000
2022-01-07 00:00:00,-0.119007,-1.000000
2022-01-08 00:00:00,-0.469093,5.000000
2022-01-09 00:00:00,0.054953,3.000000
2022-01-10 00:00:00,0.306260,3.000000


In [67]:
# mul
d5.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,0.039070,66.000000
2022-01-04 00:00:00,0.035422,6.000000
2022-01-05 00:00:00,0.059350,342.000000
2022-01-06 00:00:00,0.126948,168.000000
2022-01-07 00:00:00,0.559158,552.000000
2022-01-08 00:00:00,0.389423,14.000000
2022-01-09 00:00:00,0.706776,180.000000
2022-01-10 00:00:00,0.328877,10.000000


In [68]:
# div
d6.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,0.268489,0.545455
2022-01-04 00:00:00,0.335503,0.666667
2022-01-05 00:00:00,0.712543,0.947368
2022-01-06 00:00:00,0.706102,2.625000
2022-01-07 00:00:00,0.853012,0.958333
2022-01-08 00:00:00,0.479483,3.500000
2022-01-09 00:00:00,1.067537,1.250000
2022-01-10 00:00:00,1.695348,2.500000


### removing the NaN values from the resultant dataframe

In [69]:
# the + operation is UNION of indexes
d31 = d1+d2
# indexs not present in the other dataframe get a NaN
# do you can obvs remove nan values like before
d3.dropna().style.highlight_null(null_color='red')

,A,B
2022-01-03 00:00:00,0.483887,17.000000
2022-01-04 00:00:00,0.433941,5.000000
2022-01-05 00:00:00,0.494248,37.000000
2022-01-06 00:00:00,0.723410,29.000000
2022-01-07 00:00:00,1.500265,47.000000
2022-01-08 00:00:00,1.333319,9.000000
2022-01-09 00:00:00,1.682297,27.000000
2022-01-10 00:00:00,1.187141,7.000000


## df.align()

Pandas [align](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.align.html) works to join the dataframes in interesting ways.

In [70]:
# align returns two dataframes - left and right as a result of a join
# default join is outer - the union of indices will be used as index for left and right
left, right = d1.align(d2, join='outer')
render_df_side_by_side(left, right)

,A,B
2022-01-01 00:00:00,0.915867,7.000000
2022-01-02 00:00:00,0.204894,9.000000
2022-01-03 00:00:00,0.102420,6.000000
2022-01-04 00:00:00,0.109014,2.000000
2022-01-05 00:00:00,0.205643,18.000000
2022-01-06 00:00:00,0.299396,21.000000
2022-01-07 00:00:00,0.690629,23.000000
2022-01-08 00:00:00,0.432113,7.000000
2022-01-09 00:00:00,0.868625,15.000000
2022-01-10 00:00:00,0.746700,5.000000


In [71]:
# inner join
left, right = d1.align(d2, join='inner')
render_df_side_by_side(d1,d2)

,A,B
2022-01-01 00:00:00,0.915867,7
2022-01-02 00:00:00,0.204894,9
2022-01-03 00:00:00,0.102420,6
2022-01-04 00:00:00,0.109014,2
2022-01-05 00:00:00,0.205643,18
2022-01-06 00:00:00,0.299396,21
2022-01-07 00:00:00,0.690629,23
2022-01-08 00:00:00,0.432113,7
2022-01-09 00:00:00,0.868625,15
2022-01-10 00:00:00,0.746700,5


In [72]:
# we can fill the NaNs with a certain value if needed
# let's fill with -100 so we can see it clearly
left, right = d1.align(d2, join='outer', fill_value = -100) 
render_df_side_by_side(left, right)

,A,B
2022-01-01 00:00:00,0.915867,7
2022-01-02 00:00:00,0.204894,9
2022-01-03 00:00:00,0.102420,6
2022-01-04 00:00:00,0.109014,2
2022-01-05 00:00:00,0.205643,18
2022-01-06 00:00:00,0.299396,21
2022-01-07 00:00:00,0.690629,23
2022-01-08 00:00:00,0.432113,7
2022-01-09 00:00:00,0.868625,15
2022-01-10 00:00:00,0.746700,5


In [73]:
# left join: left index is added to right, but not the other way
left, right = d1.align(d2, join='left') 
render_df_side_by_side(left, right)

,A,B
2022-01-01 00:00:00,0.915867,7
2022-01-02 00:00:00,0.204894,9
2022-01-03 00:00:00,0.102420,6
2022-01-04 00:00:00,0.109014,2
2022-01-05 00:00:00,0.205643,18
2022-01-06 00:00:00,0.299396,21
2022-01-07 00:00:00,0.690629,23
2022-01-08 00:00:00,0.432113,7
2022-01-09 00:00:00,0.868625,15
2022-01-10 00:00:00,0.746700,5


In [74]:
# right join: right index is added to left, but not the other way
left, right = d1.align(d2, join='right') 
render_df_side_by_side(left, right)

,A,B
2022-01-03 00:00:00,0.102420,6.000000
2022-01-04 00:00:00,0.109014,2.000000
2022-01-05 00:00:00,0.205643,18.000000
2022-01-06 00:00:00,0.299396,21.000000
2022-01-07 00:00:00,0.690629,23.000000
2022-01-08 00:00:00,0.432113,7.000000
2022-01-09 00:00:00,0.868625,15.000000
2022-01-10 00:00:00,0.746700,5.000000
2022-01-11 00:00:00,nan,nan
2022-01-12 00:00:00,nan,nan


There's more to explore in align, and it can get confusing, so beginners be careful, take time and try to work out the result before you execute to build intuition.

# Operations on data

## Stats

Operations in general *exclude* missing data

In [75]:
# arithmetic mean, for each column (axis = 0)
df.mean()

A    -0.319804
B    -0.348804
C    -0.604892
D    -0.818790
F    15.500000
dtype: float64

In [76]:
# mean across a row (axis = 1)
df.mean(1)

2022-06-19    1.184350
2022-06-20    2.264479
2022-06-21    2.479823
2022-06-22    2.391746
2022-06-23    2.509441
2022-06-24    3.200988
2022-06-25    2.643692
2022-06-26    3.215380
2022-06-27    3.724142
2022-06-28    3.201380
Freq: D, dtype: float64

## Apply

In [77]:
# apply custom lambdas
df.apply(lambda x: x.max()-x.min())

A    3.330460
B    2.849929
C    2.130415
D    1.924896
F    9.000000
dtype: float64

In [78]:
df

,A,B,C,D,F
2022-06-19,-0.971039,-1.302320,-1.163174,-1.641720,11
2022-06-20,-1.018621,0.719277,-0.352777,-0.025486,12
2022-06-21,0.023506,0.267982,0.354560,-1.246931,13
2022-06-22,-0.671370,-0.802099,-0.850975,0.283176,14
2022-06-23,-0.300175,-1.309658,0.473513,-1.316474,15
2022-06-24,2.108041,0.070278,-0.961669,-1.211711,16
2022-06-25,-0.785940,-1.622560,-1.594312,0.221272,17
2022-06-26,-0.523616,-0.117206,0.189113,-1.471392,18
2022-06-27,0.163592,1.227369,-0.486297,-1.283956,19
2022-06-28,-1.222419,-0.619103,-1.656902,-0.494676,20


In [79]:
# each subsequent value is a sum of all values before it in the respective column
cumsum_df = df.apply(np.cumsum)
render_df_side_by_side(df, cumsum_df, "OG Dataframe", "Cumulative Sum")

,A,B,C,D,F
2022-06-19 00:00:00,-0.971039,-1.302320,-1.163174,-1.641720,11
2022-06-20 00:00:00,-1.018621,0.719277,-0.352777,-0.025486,12
2022-06-21 00:00:00,0.023506,0.267982,0.354560,-1.246931,13
2022-06-22 00:00:00,-0.671370,-0.802099,-0.850975,0.283176,14
2022-06-23 00:00:00,-0.300175,-1.309658,0.473513,-1.316474,15
2022-06-24 00:00:00,2.108041,0.070278,-0.961669,-1.211711,16
2022-06-25 00:00:00,-0.785940,-1.622560,-1.594312,0.221272,17
2022-06-26 00:00:00,-0.523616,-0.117206,0.189113,-1.471392,18
2022-06-27 00:00:00,0.163592,1.227369,-0.486297,-1.283956,19
2022-06-28 00:00:00,-1.222419,-0.619103,-1.656902,-0.494676,20


## Histogramming

Frequencies, 'nuff said

```value_counts()```

In [80]:
series1 = pd.Series(np.random.randint(0,5, size = 25))
series1.value_counts()

3    10
0     6
1     5
4     2
2     2
dtype: int64

## String Methods

In [81]:
# build a series
some_string = "SERIes and Index are EqUIppeD WITh A seT Of stRINg PROCESSING METHoDs In tHe ```sTR``` attrIBute THAT MakE IT eASy To oPerATE ON EACh ELEmeNt OF The aRrAY. NOtE thAt PAtTERN-maTching in ```sTR``` gENeralLY UsES reGUlAr eXpreSsiONs bY DEfault (aND In Some cases AlwayS uSEs tHeM)."
str_series = pd.Series(some_string.split()) # split splits on whitespace by default

In [82]:
# manipulating the case - lower, upper etc.
# also the length of each string, split, replace, yada yada...
# very important when cleaning data (column names may have stupid whitespace, bad case, spacing etc.)
low = str_series.str.lower()
up = str_series.str.upper()

low_up_df = pd.DataFrame(
    {
        'og': str_series,
        'length': str_series.str.len(),
        'low': low,
        'up': up,
        'split_low_on_i': low.str.split('i', expand = False), # expand = True will break this usecase, try elsewhere
        'replace_a_with_star_in_low': low.str.replace('a','*', case=False, regex=True) # yeah, you can mess with regular expns
    })

low_up_df.tail(10)

,og,length,low,up,split_low_on_i,replace_a_with_star_in_low
36,eXpreSsiONs,11,expressions,EXPRESSIONS,"[express, ons]",expressions
37,bY,2,by,BY,[by],by
38,DEfault,7,default,DEFAULT,[default],def*ult
39,(aND,4,(and,(AND,[(and],(*nd
40,In,2,in,IN,"[, n]",in
41,Some,4,some,SOME,[some],some
42,cases,5,cases,CASES,[cases],c*ses
43,AlwayS,6,always,ALWAYS,[always],*lw*ys
44,uSEs,4,uses,USES,[uses],uses
45,tHeM).,6,them).,THEM).,[them).],them).


See [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#text-string-methods) for more string operations.

# Merge

Combine series and dataframes, use set logic for indexes and relational algebra for joins/merges

## Concat

Chill, we've done this before. The same join = inner/outer/left/right drill. 

In [83]:
# let's make some more dataframes
concatDF = pd.DataFrame(np.random.randn(10,4)) # 10 rows and 4 columns of random numbers
pieces = [concatDF[:3], concatDF[3:7], concatDF[7:]]
reconstructedDF = pd.concat(pieces)
display('broken DFs: ',pieces)
render_df_side_by_side(concatDF, reconstructedDF, "OG", "Concatenated")

'broken DFs: '

[          0         1         2         3
 0  0.249482  0.657338 -0.570253 -1.371859
 1  1.411653  0.914510 -1.834488  1.393563
 2 -1.804040  1.213927  0.742795 -0.223456,
           0         1         2         3
 3 -0.387389  0.986363  0.412106  0.368848
 4 -0.064054 -1.301703 -0.152629 -0.455131
 5  1.020706 -0.377649 -0.514762  0.602171
 6 -0.710669 -0.526399 -0.970234  0.365411,
           0         1         2         3
 7 -1.080092 -0.003955 -0.186287  0.121614
 8  0.257530 -0.855749 -0.152442 -0.057362
 9 -0.834756 -0.128947 -0.251074  1.447061]

,0,1,2,3
0,0.249482,0.657338,-0.570253,-1.371859
1,1.411653,0.914510,-1.834488,1.393563
2,-1.804040,1.213927,0.742795,-0.223456
3,-0.387389,0.986363,0.412106,0.368848
4,-0.064054,-1.301703,-0.152629,-0.455131
5,1.020706,-0.377649,-0.514762,0.602171
6,-0.710669,-0.526399,-0.970234,0.365411
7,-1.080092,-0.003955,-0.186287,0.121614
8,0.257530,-0.855749,-0.152442,-0.057362
9,-0.834756,-0.128947,-0.251074,1.447061


_From the [page](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html#concat)_:
Adding a column to a DataFrame is relatively fast. However, adding a row requires a copy, and may be expensive. We recommend passing a pre-built list of records to the DataFrame constructor instead of building a DataFrame by iteratively appending records to it.

## Join
SQL style merges. 

In [84]:
# two data frames
sl = pd.Series(np.random.randint(10, size=6))
sr = sl.shift(2).fillna(0).astype(int)
left = pd.DataFrame(
    {
        'key': list('ABCDEF'),
        'lval': sl
    }
)

right = pd.DataFrame(
    {
        'key': list('ABBCDE'),
        'rval': sr
    }
)

In [85]:
render_df_side_by_side(left, right, 'left', 'right')

,key,lval
0,A,9
1,B,7
2,C,4
3,D,9
4,E,2
5,F,3
,key,rval
0,A,0
1,B,0
2,B,9


In [86]:
result = pd.merge(left, right, on='key')
result

,key,lval,rval
0,A,9,0
1,B,7,0
2,B,7,9
3,C,4,7
4,D,9,4
5,E,2,9


# Grouping

"Group By" really means one or more of the following happening:
1. **Split**: data is broken into groups based on some criteria
2. **Apply**: a data operation is executed on each of those groups
3. **Combine**: the results are combined back into a data structure

In [108]:
grp_df = pd.DataFrame(
    {
        'colA': list('AABCAABDAA'),
        'colB': list('1234567890'),
        'colC': np.random.randn(10),
        'colD': np.random.randn(10)
    }
)

# a DataFrameGroupBy object object
grpby = grp_df.groupby('colA')

# list all the groups found
g = grpby.groups
print(g)
# # create a dataframe from the group
# grpA = grp_df.groupby('colA').get_group('A')
# grpB = grp_df.groupby('colA').get_group('B')
# grpC = grp_df.groupby('colA').get_group('C')
# grpD = grp_df.groupby('colA').get_group('D')
# sumDF = grp_df.groupby('colA').sum()

# render_df_side_by_side(grpA, grpB)
# # render_df_side_by_side(grpC, grpD)
# display("sumDF: ",sumDF) #colB is missing as it's strings

{'A': [0, 1, 4, 5, 8, 9], 'B': [2, 6], 'C': [3], 'D': [7]}


In [121]:
grpby_labels_list = [*g.keys()] # using the unpacking operator *
grpby_labels_list
# alternatively
# grpby_labels_list = list(g.keys())

['A', 'B', 'C', 'D']

In [122]:
grp_list = {lbl: grpby.get_group(lbl) for lbl in grpby_labels_list}
grp_list['A']

,colA,colB,colC,colD
0,A,1,-0.139915,-0.046676
1,A,2,1.716526,0.747948
4,A,5,-0.521092,-0.536334
5,A,6,-0.390684,-0.730118
8,A,9,-0.507226,0.633689
9,A,0,0.284605,-0.373563
